In [ ]:
import tkinter as tk
from abc import ABC, abstractmethod
import copy
import time

# ================= Piece Classes =================

class Piece(ABC):
    def __init__(self, color):
        self.color = color
        self.moved = False

    @abstractmethod
    def get_valid_moves(self, board, row, col, check_for_check=True):
        pass

    def get_attack_squares(self, board, row, col):
        """Gets squares the piece attacks without check validation (to prevent recursion)"""
        return self.get_valid_moves(board, row, col, check_for_check=False)


class King(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1),         (0, 1),
                      (1, -1), (1, 0), (1, 1)]
        moves = []
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                if board.is_empty(r, c) or board.is_enemy(r, c, self.color):
                    # Add the move without checking if it puts king in check
                    if not check_for_check:
                        moves.append((r, c))
                    else:
                        # Check if move would put king in check
                        test_board = board.clone()
                        test_board.grid[r][c] = test_board.grid[row][col]
                        test_board.grid[row][col] = None
                        if not test_board.is_square_attacked(r, c, self.color):
                            moves.append((r, c))
        return moves

    def get_attack_squares(self, board, row, col):
        """Kings attack in all adjacent squares"""
        directions = [(-1, -1), (-1, 0), (-1, 1),
                     (0, -1),         (0, 1),
                     (1, -1), (1, 0), (1, 1)]
        attacks = []
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                attacks.append((r, c))
        return attacks


class Queen(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        moves = []
        # Combined moves of Rook and Bishop
        directions = [
            (-1, 0), (1, 0), (0, -1), (0, 1),  # Rook directions
            (-1, -1), (-1, 1), (1, -1), (1, 1)  # Bishop directions
        ]
        
        for dr, dc in directions:
            r, c = row, col
            while True:
                r += dr
                c += dc
                if 0 <= r < 8 and 0 <= c < 8:
                    if board.is_empty(r, c):
                        moves.append((r, c))
                    elif board.is_enemy(r, c, self.color):
                        moves.append((r, c))
                        break
                    else:
                        break
                else:
                    break
        
        return moves


class Rook(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        moves = []
        for dr, dc in directions:
            r, c = row, col
            while True:
                r += dr
                c += dc
                if 0 <= r < 8 and 0 <= c < 8:
                    if board.is_empty(r, c):
                        moves.append((r, c))
                    elif board.is_enemy(r, c, self.color):
                        moves.append((r, c))
                        break
                    else:
                        break
                else:
                    break
        return moves


class Bishop(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        moves = []
        for dr, dc in directions:
            r, c = row, col
            while True:
                r += dr
                c += dc
                if 0 <= r < 8 and 0 <= c < 8:
                    if board.is_empty(r, c):
                        moves.append((r, c))
                    elif board.is_enemy(r, c, self.color):
                        moves.append((r, c))
                        break
                    else:
                        break
                else:
                    break
        return moves


class Knight(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        directions = [(-2, -1), (-2, 1), (-1, -2), (-1, 2),
                      (1, -2), (1, 2), (2, -1), (2, 1)]
        moves = []
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                if board.is_empty(r, c) or board.is_enemy(r, c, self.color):
                    moves.append((r, c))
        return moves


class Pawn(Piece):
    def get_valid_moves(self, board, row, col, check_for_check=True):
        moves = []
        direction = -1 if self.color == 'white' else 1
        start_row = 6 if self.color == 'white' else 1

        # Forward move
        if 0 <= row + direction < 8 and board.is_empty(row + direction, col):
            moves.append((row + direction, col))
            # Double move from starting position
            if row == start_row and board.is_empty(row + 2*direction, col):
                moves.append((row + 2*direction, col))

        # Captures
        for dc in [-1, 1]:
            if 0 <= col + dc < 8 and 0 <= row + direction < 8:
                if board.is_enemy(row + direction, col + dc, self.color):
                    moves.append((row + direction, col + dc))

        return moves

    def get_attack_squares(self, board, row, col):
        """Pawns attack diagonally"""
        attacks = []
        direction = -1 if self.color == 'white' else 1
        
        # Attack squares
        for dc in [-1, 1]:
            r, c = row + direction, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                attacks.append((r, c))
                
        return attacks

# ================= Board, Move, Player Classes =================

class Move:
    def __init__(self, start_pos, end_pos, piece):
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.piece = piece
        self.captured_piece = None
        self.notation = None
        
    def set_notation(self, board):
        r1, c1 = self.start_pos
        r2, c2 = self.end_pos
        piece_type = type(self.piece).__name__
        
        # Get captured piece info before it's overwritten
        captured = board.grid[r2][c2]
        self.captured_piece = captured
        
        # Basic notation
        piece_symbols = {'King': 'K', 'Queen': 'Q', 'Rook': 'R', 'Bishop': 'B', 'Knight': 'N', 'Pawn': ''}
        files = 'abcdefgh'
        ranks = '87654321'  # Inverted because our board has 0 at the top
        
        from_square = files[c1] + ranks[r1]
        to_square = files[c2] + ranks[r2]
        
        if piece_type == 'Pawn':
            if captured:  # Capture
                self.notation = f"{files[c1]}x{to_square}"
            else:
                self.notation = to_square
        else:
            symbol = piece_symbols[piece_type]
            if captured:
                self.notation = f"{symbol}x{to_square}"
            else:
                self.notation = f"{symbol}{to_square}"
        
        return self.notation


class Board:
    def __init__(self):
        self.grid = [[None]*8 for _ in range(8)]
        self.move_history = []
        self.setup()

    def setup(self):
        for i in range(8):
            self.grid[1][i] = Pawn('black')
            self.grid[6][i] = Pawn('white')

        placement = [Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook]
        for i, piece in enumerate(placement):
            self.grid[0][i] = piece('black')
            self.grid[7][i] = piece('white')

    def is_empty(self, r, c):
        return self.grid[r][c] is None

    def is_enemy(self, r, c, color):
        return self.grid[r][c] is not None and self.grid[r][c].color != color

    def move_piece(self, move):
        r1, c1 = move.start_pos
        r2, c2 = move.end_pos
        
        # Record move
        move.set_notation(self)
        self.move_history.append(move)
        
        # Mark piece as moved (for castling, etc.)
        self.grid[r1][c1].moved = True
        
        # Execute move
        self.grid[r2][c2] = self.grid[r1][c1]
        self.grid[r1][c1] = None

    def clone(self):
        new_board = Board()
        new_board.grid = copy.deepcopy(self.grid)
        return new_board
    
    def find_king(self, color):
        for r in range(8):
            for c in range(8):
                piece = self.grid[r][c]
                if piece and isinstance(piece, King) and piece.color == color:
                    return (r, c)
        return None
    
    def is_square_attacked(self, row, col, color):
        """Check if a square is attacked by any enemy piece"""
        enemy_color = 'black' if color == 'white' else 'white'
        
        for r in range(8):
            for c in range(8):
                piece = self.grid[r][c]
                if piece and piece.color == enemy_color:
                    # Use get_attack_squares to avoid recursive check calls
                    attack_squares = piece.get_attack_squares(self, r, c)
                    if (row, col) in attack_squares:
                        return True
        return False
    
    def is_in_check(self, color):
        king_pos = self.find_king(color)
        if not king_pos:
            return False
        
        r, c = king_pos
        return self.is_square_attacked(r, c, color)
    
    def get_all_legal_moves(self, color):
        all_moves = []
        for r in range(8):
            for c in range(8):
                piece = self.grid[r][c]
                if piece and piece.color == color:
                    for move_pos in piece.get_valid_moves(self, r, c):
                        move = Move((r, c), move_pos, piece)
                        # Test move to see if it leaves king in check
                        test_board = self.clone()
                        test_board.move_piece(move)
                        if not test_board.is_in_check(color):
                            all_moves.append(move)
        return all_moves


class Player(ABC):
    def __init__(self, color):
        self.color = color
        self.captures = []
        self.move_count = 0

    @abstractmethod
    def get_move(self, game):
        pass
    
    def add_capture(self, piece):
        if piece:
            self.captures.append(piece)
    
    def record_move(self):
        self.move_count += 1


class HumanPlayer(Player):
    def get_move(self, game):
        # Will be handled by the GUI
        pass


class AIPlayer(Player):
    def __init__(self, color, depth=2):
        super().__init__(color)
        self.depth = depth
        self.last_move = None
        self.max_think_time = 2.0  # Maximum thinking time in seconds

    def get_move(self, game):
        # Set a start time to limit thinking
        start_time = time.time()
        best_score, best_move = self.minimax(game.board, self.depth, True, -float('inf'), float('inf'), start_time)
        self.last_move = best_move
        return best_move

    def minimax(self, board, depth, maximizing, alpha, beta, start_time):
        # Check if we're out of time
        if time.time() - start_time > self.max_think_time:
            return (0, None) if not maximizing else (0, self.get_any_valid_move(board))
            
        enemy_color = 'white' if self.color == 'black' else 'black'
        current_color = self.color if maximizing else enemy_color
        
        # Check for checkmate or stalemate
        legal_moves = board.get_all_legal_moves(current_color)
        if not legal_moves:
            if board.is_in_check(current_color):
                return (float('inf') if not maximizing else -float('inf')), None
            else:
                return 0, None  # Stalemate
        
        if depth == 0:
            return self.evaluate_board(board), None

        if maximizing:
            max_eval = -float('inf')
            best_move = None
            for move in legal_moves:
                clone = board.clone()
                clone.move_piece(move)
                eval_score, _ = self.minimax(clone, depth-1, False, alpha, beta, start_time)
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break
                # Check time limit
                if time.time() - start_time > self.max_think_time:
                    break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            best_move = None
            for move in legal_moves:
                clone = board.clone()
                clone.move_piece(move)
                eval_score, _ = self.minimax(clone, depth-1, True, alpha, beta, start_time)
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
                # Check time limit
                if time.time() - start_time > self.max_think_time:
                    break
            return min_eval, best_move
    
    def get_any_valid_move(self, board):
        """Fallback to get any valid move if time runs out"""
        moves = board.get_all_legal_moves(self.color)
        if moves:
            return moves[0]
        return None
    
    def evaluate_board(self, board):
        piece_values = {
            'Pawn': 1,
            'Knight': 3,
            'Bishop': 3.2,
            'Rook': 5,
            'Queen': 9,
            'King': 0  # King's value doesn't matter for material evaluation
        }
        
        # Position tables to incentivize good piece placement
        pawn_table = [
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
            [0.1, 0.1, 0.2, 0.3, 0.3, 0.2, 0.1, 0.1],
            [0.05, 0.05, 0.1, 0.25, 0.25, 0.1, 0.05, 0.05],
            [0, 0, 0, 0.2, 0.2, 0, 0, 0],
            [0.05, -0.05, -0.1, 0, 0, -0.1, -0.05, 0.05],
            [0.05, 0.1, 0.1, -0.2, -0.2, 0.1, 0.1, 0.05],
            [0, 0, 0, 0, 0, 0, 0, 0]
        ]
        
        knight_table = [
            [-0.5, -0.4, -0.3, -0.3, -0.3, -0.3, -0.4, -0.5],
            [-0.4, -0.2, 0, 0, 0, 0, -0.2, -0.4],
            [-0.3, 0, 0.1, 0.15, 0.15, 0.1, 0, -0.3],
            [-0.3, 0.05, 0.15, 0.2, 0.2, 0.15, 0.05, -0.3],
            [-0.3, 0, 0.15, 0.2, 0.2, 0.15, 0, -0.3],
            [-0.3, 0.05, 0.1, 0.15, 0.15, 0.1, 0.05, -0.3],
            [-0.4, -0.2, 0, 0.05, 0.05, 0, -0.2, -0.4],
            [-0.5, -0.4, -0.3, -0.3, -0.3, -0.3, -0.4, -0.5]
        ]
        
        # Basic evaluation: material + position
        score = 0
        for r in range(8):
            for c in range(8):
                piece = board.grid[r][c]
                if piece:
                    # Basic material value
                    value = piece_values[type(piece).__name__]
                    
                    # Position bonuses
                    if isinstance(piece, Pawn):
                        pos_value = pawn_table[r][c]
                        value += pos_value
                    elif isinstance(piece, Knight):
                        pos_value = knight_table[r][c]
                        value += pos_value
                    
                    # Control of center for minor pieces
                    if isinstance(piece, Knight) or isinstance(piece, Bishop):
                        if 2 <= r <= 5 and 2 <= c <= 5:
                            value += 0.1
                    
                    # Add or subtract value based on whose piece it is
                    if piece.color == self.color:
                        score += value
                    else:
                        score -= value
                        
        # Check for check
        enemy_color = 'white' if self.color == 'black' else 'black'
        if board.is_in_check(enemy_color):
            score += 0.5
        if board.is_in_check(self.color):
            score -= 0.5
            
        return score

# ================= Chess Game =================

class ChessGame:
    def __init__(self):
        self.board = Board()
        self.human = HumanPlayer('white')
        self.ai = AIPlayer('black', depth=2)  # Increased depth for better play
        self.current_player = self.human
        self.turn = 'white'
        self.status = 'active'  # 'active', 'checkmate', 'stalemate', 'check'
        self.winner = None
        self.move_list = []

    def switch_turn(self):
        if self.turn == 'white':
            self.turn = 'black'
            self.current_player = self.ai
        else:
            self.turn = 'white'
            self.current_player = self.human
    
    def make_move(self, move):
        # Record capture if any
        r, c = move.end_pos
        captured = self.board.grid[r][c]
        if captured:
            self.current_player.add_capture(captured)
        
        # Execute move
        self.board.move_piece(move)
        
        # Record move
        self.current_player.record_move()
        self.move_list.append(move.notation)
        
        # Check game state after move
        self.update_game_state()
        
        # Switch turn if game is still active
        if self.status == 'active' or self.status == 'check':
            self.switch_turn()
    
    def update_game_state(self):
        next_color = 'black' if self.turn == 'white' else 'white'
        legal_moves = self.board.get_all_legal_moves(next_color)
        
        if not legal_moves:
            if self.board.is_in_check(next_color):
                self.status = 'checkmate'
                self.winner = self.turn
            else:
                self.status = 'stalemate'
                self.winner = None
        elif self.board.is_in_check(next_color):
            self.status = 'check'
        else:
            self.status = 'active'

# ================= GUI Code =================

class ChessGUI:
    def __init__(self, root, game):
        self.root = root
        self.game = game
        self.selected = None
        self.valid_moves = []
        
        # Colors
        self.light_square = '#DDB88C'  # Light brown
        self.dark_square = '#8B4513'   # Dark brown
        self.selected_color = '#FFFF00'  # Yellow
        self.valid_move_color = '#90EE90'  # Light green
        self.check_color = '#FF6347'  # Tomato red
        
        # Grid for the whole layout
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(padx=10, pady=10)
        
        # Status area at top
        self.status_frame = tk.Frame(self.main_frame, bg='white', relief=tk.RAISED, bd=2)
        self.status_frame.grid(row=0, column=0, columnspan=2, sticky='we', padx=5, pady=5)
        
        self.status_label = tk.Label(self.status_frame, text="White's Turn", font=('Arial', 14, 'bold'), bg='white')
        self.status_label.pack(pady=5)
        
        # Board area on left
        self.board_frame = tk.Frame(self.main_frame)
        self.board_frame.grid(row=1, column=0, padx=5, pady=5)
        
        # Info area on right
        self.info_frame = tk.Frame(self.main_frame, bg='white', relief=tk.SUNKEN, bd=2)
        self.info_frame.grid(row=1, column=1, padx=5, pady=5, sticky='ns')
        
        # Buttons on board
        self.buttons = [[None for _ in range(8)] for _ in range(8)]
        self.create_board()
        
        # Game info display
        self.create_info_panel()
        
        # Update everything
        self.update_board()
        self.update_info_panel()

    def create_board(self):
        # Add file labels (a-h)
        for c in range(8):
            lbl = tk.Label(self.board_frame, text=chr(97+c), font=('Arial', 10))
            lbl.grid(row=8, column=c)
        
        # Add rank labels (1-8)
        for r in range(8):
            lbl = tk.Label(self.board_frame, text=str(8-r), font=('Arial', 10))
            lbl.grid(row=r, column=8)
        
        # Create the chess board
        for r in range(8):
            for c in range(8):
                btn = tk.Button(self.board_frame, width=4, height=2, font=('Arial', 18), 
                              command=lambda r=r, c=c: self.on_click(r, c))
                btn.grid(row=r, column=c)
                self.buttons[r][c] = btn

    def create_info_panel(self):
        # Player info
        self.player_frame = tk.Frame(self.info_frame, bg='white')
        self.player_frame.pack(fill=tk.X, padx=5, pady=5)
        
        tk.Label(self.player_frame, text="PLAYER (WHITE)", font=('Arial', 12, 'bold'), bg='white').pack(anchor='w')
        self.player_info = tk.Label(self.player_frame, text="Moves: 0\nCaptures: None", font=('Arial', 10), bg='white', justify=tk.LEFT)
        self.player_info.pack(anchor='w', padx=10)
        
        # AI info
        self.ai_frame = tk.Frame(self.info_frame, bg='white')
        self.ai_frame.pack(fill=tk.X, padx=5, pady=5)
        
        tk.Label(self.ai_frame, text="AI (BLACK)", font=('Arial', 12, 'bold'), bg='white').pack(anchor='w')
        self.ai_info = tk.Label(self.ai_frame, text="Moves: 0\nCaptures: None\nLast move: None", font=('Arial', 10), bg='white', justify=tk.LEFT)
        self.ai_info.pack(anchor='w', padx=10)
        
        # Move history
        self.history_frame = tk.Frame(self.info_frame, bg='white')
        self.history_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        tk.Label(self.history_frame, text="MOVE HISTORY", font=('Arial', 12, 'bold'), bg='white').pack(anchor='w')
        
        # Create a scrollable text widget for move history
        self.history_scroll = tk.Scrollbar(self.history_frame)
        self.history_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.history_text = tk.Text(self.history_frame, width=15, height=10, font=('Arial', 10), 
                                   yscrollcommand=self.history_scroll.set, bg='white')
        self.history_text.pack(fill=tk.BOTH, expand=True)
        self.history_scroll.config(command=self.history_text.yview)
        
        # Difficulty setting
        self.difficulty_frame = tk.Frame(self.info_frame, bg='white')
        self.difficulty_frame.pack(fill=tk.X, padx=5, pady=5)
        
        tk.Label(self.difficulty_frame, text="AI DIFFICULTY", font=('Arial', 12, 'bold'), bg='white').pack(anchor='w')
        
        self.difficulty_var = tk.IntVar(value=2)
        tk.Radiobutton(self.difficulty_frame, text="Easy (Depth 1)", variable=self.difficulty_var, value=1, 
                      bg='white', command=self.set_difficulty).pack(anchor='w')
        tk.Radiobutton(self.difficulty_frame, text="Medium (Depth 2)", variable=self.difficulty_var, value=2, 
                      bg='white', command=self.set_difficulty).pack(anchor='w')
        tk.Radiobutton(self.difficulty_frame, text="Hard (Depth 3)", variable=self.difficulty_var, value=3, 
                      bg='white', command=self.set_difficulty).pack(anchor='w')
        
        # New Game button
        self.new_game_btn = tk.Button(self.info_frame, text="New Game", font=('Arial', 12), 
                                     command=self.new_game)
        self.new_game_btn.pack(fill=tk.X, padx=5, pady=10)
    
    def set_difficulty(self):
        depth = self.difficulty_var.get()
        self.game.ai.depth = depth

    def on_click(self, r, c):
        # If game is over, ignore clicks
        if self.game.status in ['checkmate', 'stalemate']:
            return
            
        # If it's AI's turn, ignore clicks
        if self.game.turn == 'black':
            return
            
        if self.selected:
            r0, c0 = self.selected
            piece = self.game.board.grid[r0][c0]
            
            # Check if this is a valid move
            if (r, c) in self.valid_moves:
                move = Move((r0, c0), (r, c), piece)
                self.game.make_move(move)
                self.selected = None
                self.valid_moves = []
                self.update_board()
                self.update_info_panel()
                
                # If game is still active and it's AI's turn, make AI move
                if (self.game.status == 'active' or self.game.status == 'check') and self.game.turn == 'black':
                    self.status_label.config(text="AI is thinking...")
                    self.root.update()
                    self.root.after(100, self.make_ai_move)
            else:
                # Clicked elsewhere, deselect
                self.selected = None
                self.valid_moves = []
                self.update_board()
                
                # If clicked on another friendly piece, select it
                if self.game.board.grid[r][c] and self.game.board.grid[r][c].color == self.game.turn:
                    self.selected = (r, c)
                    self.valid_moves = []
                    piece = self.game.board.grid[r][c]
                    for move_pos in piece.get_valid_moves(self.game.board, r, c):
                        # Test if move would put king in check
                        test_board = self.game.board.clone()
                        test_move = Move((r, c), move_pos, piece)
                        test_board.move_piece(test_move)
                        
                        if not test_board.is_in_check(self.game.turn):
                            self.valid_moves.append(move_pos)
                    self.update_board()
        else:
            # Nothing selected yet, check if there's a piece here
            piece = self.game.board.grid[r][c]
            if piece and piece.color == self.game.turn:
                self.selected = (r, c)
                self.valid_moves = []
                for move_pos in piece.get_valid_moves(self.game.board, r, c):
                    # Test if move would put king in check
                    test_board = self.game.board.clone()
                    test_move = Move((r, c), move_pos, piece)
                    test_board.move_piece(test_move)
                    
                    if not test_board.is_in_check(self.game.turn):
                        self.valid_moves.append(move_pos)
                self.update_board()
                self.highlight_valid_moves()
            else:
                self.selected = None
                self.valid_moves = []
                
    def highlight_valid_moves(self):
        for r, c in self.valid_moves:
            self.buttons[r][c].config(bg=self.valid_move_color)
    
    def make_ai_move(self):
        ai_move = self.game.ai.get_move(self.game)
        if ai_move:
            self.game.make_move(ai_move)
        self.update_board()
        self.update_info_panel()
        
    def update_board(self):
        # Update all squares based on current board state
        for r in range(8):
            for c in range(8):
                # Set the background color
                if (r + c) % 2 == 0:
                    bg_color = self.light_square
                else:
                    bg_color = self.dark_square
                    
                # Highlight selected square
                if self.selected and (r, c) == self.selected:
                    bg_color = self.selected_color
                    
                # Highlight king if in check
                piece = self.game.board.grid[r][c]
                if piece and isinstance(piece, King) and piece.color == self.game.turn:
                    if self.game.board.is_in_check(self.game.turn):
                        bg_color = self.check_color
                
                # Update button background and text
                self.buttons[r][c].config(bg=bg_color)
                
                if piece:
                    piece_symbols = {
                        'King': '♔' if piece.color == 'white' else '♚',
                        'Queen': '♕' if piece.color == 'white' else '♛',
                        'Rook': '♖' if piece.color == 'white' else '♜',
                        'Bishop': '♗' if piece.color == 'white' else '♝',
                        'Knight': '♘' if piece.color == 'white' else '♞',
                        'Pawn': '♙' if piece.color == 'white' else '♟'
                    }
                    piece_type = type(piece).__name__
                    self.buttons[r][c].config(text=piece_symbols[piece_type])
                else:
                    self.buttons[r][c].config(text="")
    
    def update_info_panel(self):
        # Update status
        if self.game.status == 'checkmate':
            status_text = f"Checkmate! {self.game.winner.capitalize()} wins!"
        elif self.game.status == 'stalemate':
            status_text = "Stalemate! Game drawn."
        elif self.game.status == 'check':
            status_text = f"{self.game.turn.capitalize()} is in check!"
        else:
            status_text = f"{self.game.turn.capitalize()}'s Turn"
        self.status_label.config(text=status_text)
        
        # Update player info
        player_captures = ", ".join([type(piece).__name__ for piece in self.game.human.captures]) if self.game.human.captures else "None"
        player_info = f"Moves: {self.game.human.move_count}\nCaptures: {player_captures}"
        self.player_info.config(text=player_info)
        
        # Update AI info
        ai_captures = ", ".join([type(piece).__name__ for piece in self.game.ai.captures]) if self.game.ai.captures else "None"
        last_move = self.game.ai.last_move.notation if self.game.ai.last_move else "None"
        ai_info = f"Moves: {self.game.ai.move_count}\nCaptures: {ai_captures}\nLast move: {last_move}"
        self.ai_info.config(text=ai_info)
        
        # Update move history
        self.history_text.delete(1.0, tk.END)
        move_history = ""
        for i, move in enumerate(self.game.move_list):
            if i % 2 == 0:
                move_history += f"{i//2 + 1}. {move} "
            else:
                move_history += f"{move}\n"
        self.history_text.insert(tk.END, move_history)
    
    def new_game(self):
        self.game = ChessGame()
        self.game.ai.depth = self.difficulty_var.get()
        self.selected = None
        self.valid_moves = []
        self.update_board()
        self.update_info_panel()

# Main program
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Chess Game")
    game = ChessGame()
    gui = ChessGUI(root, game)
    root.mainloop()   